### Imports

In [1]:
import os
import praw
import openai

### Get API keys, OpenAI and Reddit

In [5]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [6]:
reddit = praw.Reddit(client_id = os.environ['client_id'], #personal key
                    client_secret= os.environ['client_secret'], #secret key
                    user_agent = 'sentiment analysis' #title
                    )

### Prompt Generation

Using reddit api to get the hot posts and top comments on the subreddit. This content will be used to pass it openai to get a sentiment Analysis of the post. 

In [18]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=10, num_comments=2, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        
        if c < skip_first:
            continue
        
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""

        submission = reddit.submission(post.id)
        title = post.title
        
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [19]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=12)

In [20]:
print(titles_and_comments[2])

Title: Fed’s Bowman says more U.S. rate hikes likely will be needed

Comments: 

Color me shocked
There will be atleast one more rate hike of 25 basis points. Will there be one more? thats the question. 

Just because the Fed stops rate hikes doesn't mean its going to start cutting them. I think this time they might just hold them for a bit till inflation is where they want it to be.



### Merging Prompt

In [21]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments


In [22]:
prompt = create_prompt(titles_and_comments[1])

In [23]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: WSJ: Pfizer’s Covid Boost Crashes to Earth. "Shares are down 31% YTD, a loss of more than $88 billion in market capitalization"

Comments: 

So what do you guys think. I have some shares and I don’t know if i should sell or hold 5 years
Surprised Pikachu face



### OpenAI API call

In [25]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment report from OpenAI: " + response["choices"][0]["text"])
    print("-"*30)

Title: WSJ: Pfizer’s Covid Boost Crashes to Earth. "Shares are down 31% YTD, a loss of more than $88 billion in market capitalization"

Comments: 

So what do you guys think. I have some shares and I don’t know if i should sell or hold 5 years
Surprised Pikachu face

Sentiment report from OpenAI: 
Company: Pfizer
Sentiment: Negative
------------------------------
Title: Fed’s Bowman says more U.S. rate hikes likely will be needed

Comments: 

Color me shocked
There will be atleast one more rate hike of 25 basis points. Will there be one more? thats the question. 

Just because the Fed stops rate hikes doesn't mean its going to start cutting them. I think this time they might just hold them for a bit till inflation is where they want it to be.

Sentiment report from OpenAI: 
No company mentioned
------------------------------
Title: Nvidia upcoming Earnings

Comments: 

Honestly I am genuinely wondering if they can post earnings to support the insane growth in their stock value
Tsmc mak